In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.3.0
      /_/

Using Python version 2.7.6 (default, Mar 22 2014 22:59:56)
SparkContext available as sc, HiveContext available as sqlCtx.


In [2]:
# исходные данные
film_id = 111
k = 50
z = 2.576

In [3]:
# файл с оценками
# user id | item id | rating | timestamp
user_marks_src = sc.textFile("/recsys/ml-100k/u.data")
user_marks_src.first()

u'196\t242\t3\t881250949'

In [4]:
# файл с фильмами
film_data_src = sc.textFile("/recsys/ml-100k/u.item")
film_data_src.first()

u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0'

In [5]:
# id, name
film_data = film_data_src.map(lambda x: (int(x.split('|')[0]), x.split('|')[1]))
film_data.take(5)

[(1, u'Toy Story (1995)'),
 (2, u'GoldenEye (1995)'),
 (3, u'Four Rooms (1995)'),
 (4, u'Get Shorty (1995)'),
 (5, u'Copycat (1995)')]

In [7]:
# user id | item id | rating | timestamp
user_marks = user_marks_src.map(lambda x: [int(i) for i in x.split('\t')])
user_marks.first()

[196, 242, 3, 881250949]

## Рассчет общего кол-ва оценок

In [8]:
# кол-во оценок на фильм (1)
from operator import add
user_films = user_marks.map(lambda x: (x[1], 1)).reduceByKey(add).sortBy(lambda x: -x[1])
user_films.take(12)

[(50, 583),
 (258, 509),
 (100, 508),
 (181, 507),
 (294, 485),
 (286, 481),
 (288, 478),
 (1, 452),
 (300, 431),
 (121, 429),
 (174, 420),
 (127, 413)]

In [9]:
# top-10 фильмов по кол-ву оценок
top10_rates = user_films.map(lambda x: x[0]).take(10)
print top10_rates

[50, 258, 100, 181, 294, 286, 288, 1, 300, 121]


##Рассчет средних оценок

In [10]:
# сумма всех оценок по фильмц
film_mark_sum = user_marks.map(lambda x: (x[1], x[2])).reduceByKey(add)
film_mark_sum.take(10)

[(2, 420),
 (4, 742),
 (6, 93),
 (8, 875),
 (10, 341),
 (12, 1171),
 (14, 726),
 (16, 125),
 (18, 28),
 (20, 246)]

In [11]:
film_avg_rating = film_mark_sum.join(user_films)

# id, (sum, count)
film_avg_rating.take(5)

[(4, (742, 209)),
 (8, (875, 219)),
 (12, (1171, 267)),
 (16, (125, 39)),
 (20, (246, 72))]

In [12]:
film_avg = film_avg_rating.map(lambda x: (x[0], float(x[1][0])/x[1][1]))

# сортировать надо по оценке и названию фильма
film_avg_with_names = film_avg.join(film_data).sortBy(lambda x: ''.join([str(5-x[1][0]), x[1][1]]))
film_avg_with_names.take(12)

[(1536, (5.0, u'Aiqing wansui (1994)')),
 (1653, (5.0, u'Entertaining Angels: The Dorothy Day Story (1996)')),
 (814, (5.0, u'Great Day in Harlem, A (1994)')),
 (1201, (5.0, u'Marlene Dietrich: Shadow and Light (1996) ')),
 (1189, (5.0, u'Prefontaine (1997)')),
 (1467, (5.0, u'Saint of Fort Washington, The (1993)')),
 (1500, (5.0, u'Santa with Muscles (1996)')),
 (1599, (5.0, u"Someone Else's America (1995)")),
 (1293, (5.0, u'Star Kid (1997)')),
 (1122, (5.0, u'They Made Me a Criminal (1939)')),
 (1449, (4.625, u'Pather Panchali (1955)')),
 (408, (4.491071428571429, u'Close Shave, A (1995)'))]

In [13]:
# top-10 фильмов по средней оценке
top10_average = film_avg_with_names.map(lambda x: x[0]).take(10)
print top10_average

[1536, 1653, 814, 1201, 1189, 1467, 1500, 1599, 1293, 1122]


## Количество человек m, оценивших фильм положительно

In [14]:
#Оценки 4 и выше -- считаются положительными
film_positive_count = user_marks.filter(lambda x: x[2] > 3).map(lambda x: (x[1], 1)).reduceByKey(add)
film_positive_count.take(5)

[(2, 51), (4, 122), (6, 15), (8, 155), (10, 59)]

## Доля человек, оценивших фильм положительно

In [15]:
film_positive_persent = film_positive_count.join(user_films).map(lambda x: (x[0], float(x[1][0])/x[1][1]))
film_positive_persent.take(10)

[(4, 0.583732057416268),
 (8, 0.7077625570776256),
 (12, 0.8689138576779026),
 (16, 0.41025641025641024),
 (20, 0.5555555555555556),
 (24, 0.5172413793103449),
 (28, 0.7246376811594203),
 (32, 0.654320987654321),
 (36, 0.07692307692307693),
 (40, 0.3333333333333333)]

## Глобальное среднее по всему датасету

In [16]:
global_avg = float(user_marks.map(lambda x: x[2]).sum()) / user_marks.map(lambda x: 1).sum()
global_avg

3.52986

##Оценки, поправленные на нехватку данных

In [17]:
# k = 50
film_avg_cor = film_avg_rating.map(lambda x: (x[0], float(x[1][0] + k*global_avg)/(x[1][1] + k))).sortBy(lambda x: -x[1])
film_avg_cor.take(12)

[(318, 4.331876436781609),
 (64, 4.307786786786787),
 (483, 4.298610921501706),
 (50, 4.293037914691943),
 (12, 4.250766561514196),
 (603, 4.221980694980695),
 (98, 4.203393181818182),
 (127, 4.2019287257019435),
 (408, 4.194401234567901),
 (169, 4.187458333333334),
 (174, 4.175517021276596),
 (357, 4.170359872611464)]

In [18]:
# top-10
top10_rating = film_avg_cor.map(lambda x: x[0]).take(10)
print top10_rating

[318, 64, 483, 50, 12, 603, 98, 127, 408, 169]


## Нижняя и верхняя границыдоверительного интервала оценки

In [22]:
#  где  n - количество рейтингов (пункт 1), p - доля людей, оценивших фильм положительно (п. 4).

import math
#z = 2.576
def wilson(n, p, z, sign):
    x1 = 1. / (1 + (z**2)/n)
    x2 = p + (z**2)/(2*n) 
    x3 = p * (1 - p) / n
    x4 = (z**2)/(4 * (n**2))
    return x1 * (x2 + sign * z * math.sqrt(x3 + x4))


wilson_min = user_films.join(film_positive_persent)\
    .map(lambda x: (x[0], wilson(x[1][0], x[1][1], z, -1)))\
    .sortBy(lambda x: -x[1])
wilson_min.take(12)

[(64, 0.8457526835447277),
 (318, 0.8336980984637925),
 (98, 0.8334527970235317),
 (479, 0.8332601854479507),
 (483, 0.8262807973523965),
 (50, 0.8182013061987115),
 (603, 0.8161443229080241),
 (427, 0.8134728086457121),
 (357, 0.808875556038692),
 (12, 0.8066550869388946),
 (127, 0.7990899541880941),
 (408, 0.7945044582653329)]

In [23]:
top10_lower = wilson_min.map(lambda x: x[0]).take(10)
print top10_lower

[64, 318, 98, 479, 483, 50, 603, 427, 357, 12]


## Распределение оценок

In [25]:
# распределение оценок для заданного фильма
hist_film_c = user_marks.filter(lambda x: x[1] is film_id).map(lambda x: (x[2],1)).reduceByKey(add).sortBy(lambda x: x[0])
hist_film_c.collect()

[(1, 14), (2, 18), (3, 92), (4, 118), (5, 30)]

In [28]:
hist_film = hist_film_c.map(lambda x: x[1]).take(5)
print hist_film

[14, 18, 92, 118, 30]


In [29]:
# распределение оценок для всех фильмов
hist_film_all = user_marks.map(lambda x: (x[2],1)).reduceByKey(add).sortBy(lambda x: x[0])
hist_film_all.collect()

[(1, 6110), (2, 11370), (3, 27145), (4, 34174), (5, 21201)]

In [30]:
hist_all = hist_film_all.map(lambda x: x[1]).take(5)
print hist_all

[6110, 11370, 27145, 34174, 21201]


In [32]:
# сохраняем
import json

result = {
    "top10_rates": top10_rates,
    "top10_average": top10_average,
    "top10_rating": top10_rating,
    "top10_lower": top10_lower,
    "hist_film": hist_film,
    "hist_all": hist_all
}

with open('lab9.json', 'w') as f:
    f.write(json.dumps(result))
    f.close()